In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/users/hadiahmed/documents/projects/football-predictor/src')

In [4]:
from football_model.data.get_data import get_understat_data
from football_model.features.add_metadata import add_rounds_to_data, add_home_away_goals_xg, add_match_ids
from football_model.data.prepare_model_data import prepare_model_data
from football_model.model.model import build_model
from football_model.types.model_data import ModelConfig

In [ ]:
## Model is trained on 2024-2025 season data to assess fit (will be deployed on 2025-2026 season data)

## Pull in data and prepare model input
df = get_understat_data(leagues=['EPL'],years=[str(i) for i in range(2024,2025)])
df = add_rounds_to_data(df)
df = add_match_ids(df)
df = add_home_away_goals_xg(df)
input_model_data = prepare_model_data(df,max_round=30)

# Create model configuration
config = ModelConfig(
    clip_theta=5.0,
    center_team_strength=False,
)

# Build the model
model = build_model(input_model_data, config)

/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [18]:
from datetime import datetime
df[df['round']>=30]

,datetime,team,opp_team,team_long,opp_team_long,goals,xG,goals_against,xGA,is_home,...,season,gd,round,cum_round,match_key,match_id,goals_home,xG_home,goals_away,xG_away
660,2025-04-22 19:00:00,AVL,MCI,Aston Villa,Manchester City,1,1.859060,2,1.362320,0,...,2024,-1,30,30,2025-04-22 19:00:00_AVL_MCI,330,1.0,1.859060,2.0,1.362320
661,2025-04-22 19:00:00,MCI,AVL,Manchester City,Aston Villa,2,1.362320,1,1.859060,1,...,2024,1,30,30,2025-04-22 19:00:00_AVL_MCI,330,2.0,1.362320,1.0,1.859060
662,2025-04-23 19:00:00,ARS,CRY,Arsenal,Crystal Palace,2,1.921490,2,2.505150,1,...,2024,0,30,30,2025-04-23 19:00:00_ARS_CRY,331,2.0,1.921490,2.0,2.505150
663,2025-04-23 19:00:00,CRY,ARS,Crystal Palace,Arsenal,2,2.505150,2,1.921490,0,...,2024,0,30,30,2025-04-23 19:00:00_ARS_CRY,331,2.0,2.505150,2.0,1.921490
664,2025-04-26 11:30:00,CHE,EVE,Chelsea,Everton,1,0.768313,0,0.916838,1,...,2024,1,30,30,2025-04-26 11:30:00_CHE_EVE,332,1.0,0.768313,0.0,0.916838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,2025-05-25 15:00:00,AVL,MUN,Aston Villa,Manchester United,0,0.335222,2,2.962590,0,...,2024,-2,34,34,2025-05-25 15:00:00_AVL_MUN,371,0.0,0.335222,2.0,2.962590
741,2025-05-25 15:00:00,EVE,NEW,Everton,Newcastle United,1,1.072790,0,1.159100,0,...,2024,1,34,34,2025-05-25 15:00:00_EVE_NEW,377,1.0,1.072790,0.0,1.159100
740,2025-05-25 15:00:00,CHE,NOT,Chelsea,Nottingham Forest,1,1.308740,0,1.580140,0,...,2024,1,34,34,2025-05-25 15:00:00_CHE_NOT,375,1.0,1.308740,0.0,1.580140
748,2025-05-25 15:00:00,FLH,MCI,Fulham,Manchester City,0,2.217970,2,2.459230,1,...,2024,-2,34,34,2025-05-25 15:00:00_FLH_MCI,378,0.0,2.217970,2.0,2.459230


In [ ]:
## Inspect model parameters

model

  sigma_att ~ HalfNormal(0, 0.008)
    rho_att ~ Beta(29, 1)
  sigma_def ~ HalfNormal(0, 0.008)
    rho_def ~ Beta(29, 1)
      att_0 ~ Normal(0, 0.2)
      def_0 ~ Normal(0, 0.2)
 att_rw_std ~ Normal(0, 1)
 def_rw_std ~ Normal(0, 1)
    home_mu ~ Normal(0.13, 0.03)
    home_sd ~ HalfNormal(0, 0.02)
   home_adv ~ Normal(home_mu, home_sd)
     att_rw ~ Deterministic(f(sigma_att, rho_att, att_rw_std))
     def_rw ~ Deterministic(f(sigma_def, rho_def, def_rw_std))
     attack ~ Deterministic(f(att_0, sigma_att, rho_att, att_rw_std))
    defence ~ Deterministic(f(def_0, sigma_def, rho_def, def_rw_std))
lambda_home ~ Deterministic(f(home_adv, def_0, att_0, sigma_def, rho_def, def_rw_std, sigma_att, rho_att, att_rw_std))
lambda_away ~ Deterministic(f(def_0, att_0, sigma_def, rho_def, def_rw_std, sigma_att, rho_att, att_rw_std))
 goals_home ~ Poisson(lambda_home)
 goals_away ~ Poisson(lambda_away)

In [ ]:
import pymc as pm

# Sample from the posterior with better settings to handle divergences
# Increased for 2-season model (67 time steps = more parameters)
with model:
    trace = pm.sample(
        5000,  
        tune=2000,  
        chains=4,  
        target_accept=0.97,  
        random_seed=42,
        idata_kwargs={"log_likelihood": True},
        return_inferencedata=True,
        discard_tuned_samples=True
    )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv]


Output()

Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 727 seconds.
There were 734 divergences after tuning. Increase `target_accept` or reparameterize.
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [11]:
# Check convergence diagnostics
import arviz as az

# Summary statistics and convergence diagnostics
summary = az.summary(trace, round_to=3)
print(summary[summary['r_hat'] > 1.01])  # Show problematic parameters

Empty DataFrame
Columns: [mean, sd, hdi_3%, hdi_97%, mcse_mean, mcse_sd, ess_bulk, ess_tail, r_hat]
Index: []


In [26]:
# Check if tuning was sufficient by looking at acceptance rates and step sizes
import arviz as az

# Sample stats tell you about the sampler behavior
sample_stats = trace.sample_stats

# Check final acceptance rates (should be close to target_accept=0.95)
acceptance_rate = sample_stats.acceptance_rate.mean().values
print(f"Mean acceptance rate: {acceptance_rate:.3f} (target: 0.95)")

# Check step sizes stabilized (look at the last tune iterations)
print(f"\nStep size per chain: {sample_stats.step_size.isel(draw=-1).values}")

# Check divergences
n_divergences = sample_stats.diverging.sum().values
print(f"\nTotal divergences: {n_divergences}")

Mean acceptance rate: 0.935 (target: 0.95)

Step size per chain: [0.05888671 0.07735848 0.0696604  0.03869496]

Total divergences: 524


In [12]:
# Sample from the posterior predictive distribution
with model:
    posterior_predictive = pm.sample_posterior_predictive(trace, random_seed=42)

Sampling: [goals_away, goals_home]


Output()

## In Sample MAE

In [ ]:


# Calculate prediction metrics
import numpy as np

# Get mean predictions
goals_home_pred = posterior_predictive.posterior_predictive['goals_home'].mean(dim=['chain', 'draw']).values
goals_away_pred = posterior_predictive.posterior_predictive['goals_away'].mean(dim=['chain', 'draw']).values

# Calculate MAE
mae_home = np.mean(np.abs(goals_home_pred - input_model_data.goals_home))
mae_away = np.mean(np.abs(goals_away_pred - input_model_data.goals_away))

print(f"MAE (Home Goals): {mae_home:.3f}")
print(f"MAE (Away Goals): {mae_away:.3f}")
print(f"Overall MAE: {(mae_home + mae_away)/2:.3f}")

MAE (Home Goals): 0.874
MAE (Away Goals): 0.866
Overall MAE: 0.870


## OOS Performance MAE (on 2 gameweeks)

In [15]:
# Evaluate on OUT-OF-SAMPLE test data (rounds 33+)
# Prepare test data
test_data = prepare_model_data(df, test_round=100)  # Use all available data

# Get test indices (only rounds > 0)
test_mask = df['cum_round'] > 29
test_df = df[test_mask].reset_index(drop=True)

print(f"Training data: {len(input_model_data.goals_home)} matches (rounds 1-{test_df['round'].min()})")
print(f"Test data: {len(test_df)//2} matches (rounds {test_df['round'].min()+1}+)")  # Divide by 2 because each match is 2 rows

Training data: 678 matches (rounds 1-30)
Test data: 50 matches (rounds 31+)


In [ ]:
import pandas as pd
# Make predictions on test data using the trained model
# First, prepare the full dataset to get team_id mappings
df_full = get_understat_data(leagues=['EPL'],years=['2024'])
df_full = add_rounds_to_data(df_full)
df_full = add_match_ids(df_full)
df_full = add_home_away_goals_xg(df_full)

# Add team_id mappings (same as in prepare_model_data)
teams = pd.unique(df_full[['team', 'opp_team']].values.ravel())
team_idx_map = {t: i for i, t in enumerate(teams)}
df_full["team_id"] = df_full["team"].map(team_idx_map)
df_full["opp_id"] = df_full["opp_team"].map(team_idx_map)

# Extract test data indices
test_mask = (df_full['cum_round'] > 29).values
test_indices = np.where(test_mask)[0]

# Get posterior means from the trained model
attack_mean = trace.posterior['attack'].mean(dim=['chain', 'draw']).values
defense_mean = trace.posterior['defence'].mean(dim=['chain', 'draw']).values
home_adv_mean = trace.posterior['home_adv'].mean(dim=['chain', 'draw']).values
# beta_xG_mean = trace.posterior['beta_xG'].mean(dim=['chain', 'draw']).values

# print(f"Learned beta_xG weight: {beta_xG_mean:.3f}")
print(f"(1.0 means full trust in xG, <1.0 means down-weighted)")

# Build predictions for test set
last_t = attack_mean.shape[0] - 1

test_lambda_home = []
test_lambda_away = []

for idx in test_indices:
    team_id = int(df_full.iloc[idx]['team_id'])
    opp_id = int(df_full.iloc[idx]['opp_id'])
    is_home = df_full.iloc[idx]['is_home']
    xG_home = df_full.iloc[idx]['xG_home']
    xG_away = df_full.iloc[idx]['xG_away']
    
    # Use weighted xG baseline
    theta_home = np.log(xG_home + 0.01) + attack_mean[last_t, team_id] - defense_mean[last_t, opp_id]
    if is_home == 1:
        theta_home += home_adv_mean[team_id]
    
    theta_away = np.log(xG_away + 0.01) + attack_mean[last_t, opp_id] - defense_mean[last_t, team_id]
    
    test_lambda_home.append(np.exp(np.clip(theta_home, -5, 5)))
    test_lambda_away.append(np.exp(np.clip(theta_away, -5, 5)))

test_lambda_home = np.array(test_lambda_home)
test_lambda_away = np.array(test_lambda_away)

# Get actual test goals
test_goals_home = df_full[test_mask]['goals_home'].values
test_goals_away = df_full[test_mask]['goals_away'].values

# Calculate OOS MAE
oos_mae_home = np.mean(np.abs(test_lambda_home - test_goals_home))
oos_mae_away = np.mean(np.abs(test_lambda_away - test_goals_away))

print(f"\n=== OUT-OF-SAMPLE PERFORMANCE ===")
print(f"OOS MAE (Home Goals): {oos_mae_home:.3f}")
print(f"OOS MAE (Away Goals): {oos_mae_away:.3f}")
print(f"OOS Overall MAE: {(oos_mae_home + oos_mae_away)/2:.3f}")

(1.0 means full trust in xG, <1.0 means down-weighted)

=== OUT-OF-SAMPLE PERFORMANCE ===
OOS MAE (Home Goals): 1.286
OOS MAE (Away Goals): 1.173
OOS Overall MAE: 1.230


/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

## OOS Log Likelihood metrics

In [43]:
# Calculate log likelihood for OOS predictions
from scipy.stats import poisson

# Our model predictions (using Poisson now)
model_ll_home = poisson.logpmf(test_goals_home.astype(int), test_lambda_home).sum()
model_ll_away = poisson.logpmf(test_goals_away.astype(int), test_lambda_away).sum()
model_ll_total = model_ll_home + model_ll_away
model_ll_per_goal = model_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\n=== OUT-OF-SAMPLE LOG LIKELIHOOD ===")
print(f"Our Model Total LL: {model_ll_total:.2f}")
print(f"Our Model LL per goal: {model_ll_per_goal:.3f}")

# Benchmark 1: Poisson with historical average (naive baseline)
historical_avg_home = df_full[df_full['cum_round'] <= 32]['goals_home'].mean()
historical_avg_away = df_full[df_full['cum_round'] <= 32]['goals_away'].mean()

naive_ll_home = poisson.logpmf(test_goals_home.astype(int), historical_avg_home).sum()
naive_ll_away = poisson.logpmf(test_goals_away.astype(int), historical_avg_away).sum()
naive_ll_total = naive_ll_home + naive_ll_away
naive_ll_per_goal = naive_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\nNaive Baseline (historical avg) Total LL: {naive_ll_total:.2f}")
print(f"Naive Baseline LL per goal: {naive_ll_per_goal:.3f}")

# Benchmark 2: Team-specific averages (no time dynamics, no home advantage)
team_goals_scored = df_full[df_full['cum_round'] <= 32].groupby('team_id')['goals_home'].mean()
team_goals_conceded = df_full[df_full['cum_round'] <= 32].groupby('team_id')['goals_away'].mean()

simple_ll_home = []
simple_ll_away = []

for idx in test_indices:
    team_id = int(df_full.iloc[idx]['team_id'])
    opp_id = int(df_full.iloc[idx]['opp_id'])
    
    # Simple prediction: team's avg scored vs opp's avg conceded
    lambda_h = (team_goals_scored.get(team_id, historical_avg_home) + 
                team_goals_conceded.get(opp_id, historical_avg_away)) / 2
    lambda_a = (team_goals_scored.get(opp_id, historical_avg_away) + 
                team_goals_conceded.get(team_id, historical_avg_home)) / 2
    
    simple_ll_home.append(poisson.logpmf(int(df_full.iloc[idx]['goals_home']), lambda_h))
    simple_ll_away.append(poisson.logpmf(int(df_full.iloc[idx]['goals_away']), lambda_a))

simple_ll_total = np.sum(simple_ll_home) + np.sum(simple_ll_away)
simple_ll_per_goal = simple_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\nSimple Team Averages Total LL: {simple_ll_total:.2f}")
print(f"Simple Team Averages LL per goal: {simple_ll_per_goal:.3f}")

# Calculate improvements
print(f"\n=== IMPROVEMENTS OVER BASELINES ===")
print(f"vs Naive: {model_ll_total - naive_ll_total:.2f} total LL improvement")
print(f"vs Simple Team Avg: {model_ll_total - simple_ll_total:.2f} total LL improvement")
print(f"\nBayes Factor vs Naive: {np.exp(model_ll_total - naive_ll_total):.2e}")
print(f"Bayes Factor vs Simple: {np.exp(model_ll_total - simple_ll_total):.2e}")


=== OUT-OF-SAMPLE LOG LIKELIHOOD ===
Our Model Total LL: -262.51
Our Model LL per goal: -1.601

Naive Baseline (historical avg) Total LL: -236.10
Naive Baseline LL per goal: -1.440

Simple Team Averages Total LL: -233.68
Simple Team Averages LL per goal: -1.425

=== IMPROVEMENTS OVER BASELINES ===
vs Naive: -26.41 total LL improvement
vs Simple Team Avg: -28.83 total LL improvement

Bayes Factor vs Naive: 3.38e-12
Bayes Factor vs Simple: 3.01e-13


In [44]:
# Debug: Check alpha value and compare Poisson vs NB
# print(f"Alpha (dispersion) mean: {alpha_mean:.4f}")
# print(f"This means variance = mean * (1 + alpha * mean)")

# Compare our NB predictions vs simple Poisson on same lambdas
print("\n=== Comparing likelihood functions ===")

# Use Poisson instead of NB for fair comparison with baselines
model_poisson_ll_home = poisson.logpmf(test_goals_home.astype(int), test_lambda_home).sum()
model_poisson_ll_away = poisson.logpmf(test_goals_away.astype(int), test_lambda_away).sum()
model_poisson_total = model_poisson_ll_home + model_poisson_ll_away

print(f"\nOur Model with Poisson LL: {model_poisson_total:.2f}")
print(f"Our Model with NegBin LL: {model_ll_total:.2f}")
print(f"Naive Baseline (Poisson): {naive_ll_total:.2f}")
print(f"Simple Team Avg (Poisson): {simple_ll_total:.2f}")

print(f"\n=== FAIR COMPARISON (all using Poisson) ===")
print(f"Our Model vs Naive: {model_poisson_total - naive_ll_total:.2f} LL improvement")
print(f"Our Model vs Simple: {model_poisson_total - simple_ll_total:.2f} LL improvement")
print(f"\nBayes Factor vs Naive: {np.exp(model_poisson_total - naive_ll_total):.2e}")
print(f"Bayes Factor vs Simple: {np.exp(model_poisson_total - simple_ll_total):.2e}")

# Show why our lambdas might be worse
print(f"\n=== Lambda Statistics ===")
print(f"Our model - Home lambda mean: {test_lambda_home.mean():.3f}, std: {test_lambda_home.std():.3f}")
print(f"Our model - Away lambda mean: {test_lambda_away.mean():.3f}, std: {test_lambda_away.std():.3f}")
print(f"Naive baseline - Home: {historical_avg_home:.3f}, Away: {historical_avg_away:.3f}")
print(f"\nActual test data - Home mean: {test_goals_home.mean():.3f}, Away mean: {test_goals_away.mean():.3f}")


=== Comparing likelihood functions ===

Our Model with Poisson LL: -262.51
Our Model with NegBin LL: -262.51
Naive Baseline (Poisson): -236.10
Simple Team Avg (Poisson): -233.68

=== FAIR COMPARISON (all using Poisson) ===
Our Model vs Naive: -26.41 LL improvement
Our Model vs Simple: -28.83 LL improvement

Bayes Factor vs Naive: 3.38e-12
Bayes Factor vs Simple: 3.01e-13

=== Lambda Statistics ===
Our model - Home lambda mean: 0.635, std: 0.357
Our model - Away lambda mean: 0.590, std: 0.324
Naive baseline - Home: 1.472, Away: 1.472

Actual test data - Home mean: 1.305, Away mean: 1.305


In [45]:
# Check which parameters have worst convergence
import pandas as pd

worst_rhat = summary.nlargest(20, 'r_hat')[['mean', 'sd', 'r_hat']]
print("Worst 20 R-hat values:")
print(worst_rhat)

# Check if it's specific to certain parameter types
print("\n=== R-hat by parameter type ===")
for param_type in ['attack', 'defense', 'home_adv', 'match_effect', 'rho', 'sigma']:
    param_mask = summary.index.str.contains(param_type)
    if param_mask.any():
        bad_count = (summary.loc[param_mask, 'r_hat'] > 1.01).sum()
        total_count = param_mask.sum()
        print(f"{param_type}: {bad_count}/{total_count} with r_hat > 1.01")

Worst 20 R-hat values:
                     mean     sd  r_hat
home_sd             0.017  0.012  1.011
def_rw_std[30, 7]   0.050  1.039  1.005
att_rw_std[2, 17]   0.337  0.970  1.004
att_rw_std[3, 17]   0.384  0.990  1.004
att_rw_std[6, 9]   -0.114  1.006  1.004
att_rw_std[10, 6]  -0.512  0.996  1.004
att_rw_std[12, 5]   0.205  0.992  1.004
att_rw_std[21, 15] -0.607  0.997  1.004
def_rw_std[2, 15]  -0.069  1.045  1.004
att_0[7]            0.076  0.176  1.003
att_rw_std[1, 19]  -0.136  1.008  1.003
att_rw_std[8, 2]    0.321  0.987  1.003
att_rw_std[9, 2]    0.290  0.986  1.003
att_rw_std[9, 4]   -0.136  0.981  1.003
att_rw_std[17, 15] -0.444  0.981  1.003
att_rw_std[19, 5]  -0.761  1.025  1.003
att_rw_std[21, 3]  -0.977  0.994  1.003
att_rw_std[26, 5]  -0.648  1.012  1.003
att_rw_std[27, 10] -0.578  0.997  1.003
att_rw_std[30, 19] -0.204  0.985  1.003

=== R-hat by parameter type ===
attack: 0/660 with r_hat > 1.01
home_adv: 0/20 with r_hat > 1.01
rho: 0/2 with r_hat > 1.01
sigma: 0/2 w

In [21]:
# Final convergence check
print("=== CONVERGENCE SUMMARY ===")
print(f"Total parameters with R-hat > 1.01: {(summary['r_hat'] > 1.01).sum()}/{len(summary)}")
print(f"Total parameters with R-hat > 1.05: {(summary['r_hat'] > 1.05).sum()}/{len(summary)}")

print("\nWorst R-hat values:")
print(summary.nlargest(10, 'r_hat')[['mean', 'sd', 'r_hat']])

print("\n=== KEY PARAMETERS ===")
for param in ['sigma_att', 'sigma_def', 'rho_att', 'rho_def', 'beta_xG']:
    if param in summary.index:
        row = summary.loc[param]
        print(f"{param}: mean={row['mean']:.4f}, r_hat={row['r_hat']:.3f}")

=== CONVERGENCE SUMMARY ===
Total parameters with R-hat > 1.01: 722/5143
Total parameters with R-hat > 1.05: 0/5143

Worst R-hat values:
                     mean     sd  r_hat
def_rw_std[9, 1]    0.130  1.144  1.040
att_rw_std[5, 18]   0.058  1.235  1.032
def_rw_std[1, 2]   -0.106  1.095  1.031
att_rw_std[6, 14]   0.168  1.160  1.029
def_rw_std[14, 19] -0.261  1.209  1.029
def_rw_std[18, 7]   0.114  1.175  1.029
def_rw_std[9, 16]  -0.156  1.121  1.028
def_rw_std[23, 16]  0.176  1.113  1.028
def_rw_std[30, 12]  0.093  1.108  1.028
lambda_home[191]    1.557  0.292  1.028

=== KEY PARAMETERS ===
sigma_att: mean=0.0090, r_hat=1.003
sigma_def: mean=0.0080, r_hat=1.009
rho_att: mean=0.9810, r_hat=1.005
rho_def: mean=0.9530, r_hat=1.016
beta_xG: mean=0.1990, r_hat=1.010


# Rolling Window Cross-Validation

Evaluate model robustness by training on expanding windows and testing on future rounds.

In [19]:
# Rolling window cross-validation
import pandas as pd
from scipy.stats import poisson
import pymc as pm
import numpy as np
# Configuration
min_train_rounds = 25  # Minimum rounds to train on
test_window = 1        # Test on next 5 rounds
step_size = 1          # Move forward by 5 rounds each time

# Get full dataset
df_cv = get_understat_data(leagues=['EPL'], years=['2024'])
df_cv = add_rounds_to_data(df_cv)
df_cv = add_match_ids(df_cv)
df_cv = add_home_away_goals_xg(df_cv)

max_round = df_cv['cum_round'].max()
print(f"Total rounds available: {max_round}")

# Define windows
windows = []
current_train_end = min_train_rounds
while current_train_end + test_window <= max_round:
    test_start = current_train_end + 1
    test_end = min(test_start + test_window - 1, max_round)
    windows.append({
        'train_start': 1,
        'train_end': current_train_end,
        'test_start': test_start,
        'test_end': test_end
    })
    current_train_end += step_size

print(f"\n=== CROSS-VALIDATION WINDOWS ===")
for i, w in enumerate(windows, 1):
    n_train = len(df_cv[df_cv['cum_round'] <= w['train_end']]) // 2
    n_test = len(df_cv[(df_cv['cum_round'] >= w['test_start']) & 
                       (df_cv['cum_round'] <= w['test_end'])]) // 2
    print(f"Window {i}: Train rounds {w['train_start']}-{w['train_end']} ({n_train} matches) → "
          f"Test rounds {w['test_start']}-{w['test_end']} ({n_test} matches)")

Total rounds available: 34

=== CROSS-VALIDATION WINDOWS ===
Window 1: Train rounds 1-25 (281 matches) → Test rounds 26-26 (8 matches)
Window 2: Train rounds 1-26 (289 matches) → Test rounds 27-27 (20 matches)
Window 3: Train rounds 1-27 (309 matches) → Test rounds 28-28 (10 matches)
Window 4: Train rounds 1-28 (319 matches) → Test rounds 29-29 (11 matches)
Window 5: Train rounds 1-29 (330 matches) → Test rounds 30-30 (9 matches)
Window 6: Train rounds 1-30 (339 matches) → Test rounds 31-31 (11 matches)
Window 7: Train rounds 1-31 (350 matches) → Test rounds 32-32 (10 matches)
Window 8: Train rounds 1-32 (360 matches) → Test rounds 33-33 (8 matches)
Window 9: Train rounds 1-33 (368 matches) → Test rounds 34-34 (12 matches)


/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [22]:
# Run cross-validation (this will take time!)
results = []

for i, window in enumerate(windows, 1):
    print(f"\n{'='*60}")
    print(f"WINDOW {i}/{len(windows)}: Training on rounds {window['train_start']}-{window['train_end']}")
    print(f"{'='*60}")
    
    # Prepare training data
    train_data = prepare_model_data(df_cv, test_round=window['train_end'])
    
    # Build and sample model
    config = ModelConfig(clip_theta=5.0, center_team_strength=False, use_opponent_adjusted_xG=True,use_xG=True)
    model_cv = build_model(train_data, config)
    
    with model_cv:
        trace_cv = pm.sample(
            2000,  # Fewer samples for speed
            tune=2000,
            chains=3,  # Fewer chains for speed
            target_accept=0.95,
            random_seed=42 + i,
            return_inferencedata=True,
            discard_tuned_samples=True
        )
    
    # Get posterior means
    attack_mean_cv = trace_cv.posterior['attack'].mean(dim=['chain', 'draw']).values
    defense_mean_cv = trace_cv.posterior['defence'].mean(dim=['chain', 'draw']).values
    home_adv_mean_cv = trace_cv.posterior['home_adv'].mean(dim=['chain', 'draw']).values
    beta_xG_mean_cv = trace_cv.posterior['beta_xG'].mean(dim=['chain', 'draw']).values
    
    # Prepare test data
    test_mask_cv = ((df_cv['cum_round'] >= window['test_start']) & 
                    (df_cv['cum_round'] <= window['test_end'])).values
    test_indices_cv = np.where(test_mask_cv)[0]
    
    # Team mapping
    teams_cv = pd.unique(df_cv[['team', 'opp_team']].values.ravel())
    team_idx_map_cv = {t: i for i, t in enumerate(teams_cv)}
    df_cv["team_id"] = df_cv["team"].map(team_idx_map_cv)
    df_cv["opp_id"] = df_cv["opp_team"].map(team_idx_map_cv)
    
    # Make predictions
    last_t_cv = attack_mean_cv.shape[0] - 1
    test_lambda_home_cv = []
    test_lambda_away_cv = []
    
    for idx in test_indices_cv:
        team_id = int(df_cv.iloc[idx]['team_id'])
        opp_id = int(df_cv.iloc[idx]['opp_id'])
        is_home = df_cv.iloc[idx]['is_home']
        xG_home = df_cv.iloc[idx]['xG_home']
        xG_away = df_cv.iloc[idx]['xG_away']
        
        theta_home = (beta_xG_mean_cv * np.log(xG_home + 0.01) + 
                      attack_mean_cv[last_t_cv, team_id] - 
                      defense_mean_cv[last_t_cv, opp_id])
        if is_home == 1:
            theta_home += home_adv_mean_cv[team_id]
        
        theta_away = (beta_xG_mean_cv * np.log(xG_away + 0.01) + 
                      attack_mean_cv[last_t_cv, opp_id] - 
                      defense_mean_cv[last_t_cv, team_id])
        
        test_lambda_home_cv.append(np.exp(np.clip(theta_home, -5, 5)))
        test_lambda_away_cv.append(np.exp(np.clip(theta_away, -5, 5)))
    
    test_lambda_home_cv = np.array(test_lambda_home_cv)
    test_lambda_away_cv = np.array(test_lambda_away_cv)
    test_goals_home_cv = df_cv[test_mask_cv]['goals_home'].values
    test_goals_away_cv = df_cv[test_mask_cv]['goals_away'].values
    
    # Calculate metrics
    mae_home_cv = np.mean(np.abs(test_lambda_home_cv - test_goals_home_cv))
    mae_away_cv = np.mean(np.abs(test_lambda_away_cv - test_goals_away_cv))
    mae_cv = (mae_home_cv + mae_away_cv) / 2
    
    ll_home_cv = poisson.logpmf(test_goals_home_cv.astype(int), test_lambda_home_cv).sum()
    ll_away_cv = poisson.logpmf(test_goals_away_cv.astype(int), test_lambda_away_cv).sum()
    ll_total_cv = ll_home_cv + ll_away_cv
    
    # Naive baseline
    naive_lambda = df_cv[df_cv['cum_round'] <= window['train_end']]['goals_home'].mean()
    naive_ll_cv = (poisson.logpmf(test_goals_home_cv.astype(int), naive_lambda).sum() +
                   poisson.logpmf(test_goals_away_cv.astype(int), naive_lambda).sum())
    
    results.append({
        'window': i,
        'train_rounds': f"{window['train_start']}-{window['train_end']}",
        'test_rounds': f"{window['test_start']}-{window['test_end']}",
        'n_train': len(train_data.goals_home),
        'n_test': len(test_goals_home_cv) + len(test_goals_away_cv),
        'mae': mae_cv,
        'log_likelihood': ll_total_cv,
        'll_naive': naive_ll_cv,
        'll_improvement': ll_total_cv - naive_ll_cv,
        'beta_xG': beta_xG_mean_cv
    })
    
    print(f"Window {i} Results:")
    print(f"  MAE: {mae_cv:.3f}")
    print(f"  Log Likelihood: {ll_total_cv:.2f} (vs Naive: {naive_ll_cv:.2f})")
    print(f"  Improvement: {ll_total_cv - naive_ll_cv:.2f}")

print(f"\n{'='*60}")
print("CROSS-VALIDATION COMPLETE")
print(f"{'='*60}")


WINDOW 1/9: Training on rounds 1-25


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 3 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]


Output()

Sampling 3 chains for 2_000 tune and 2_000 draw iterations (6_000 + 6_000 draws total) took 170 seconds.
There were 341 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Window 1 Results:
  MAE: 0.750
  Log Likelihood: -44.54 (vs Naive: -46.80)
  Improvement: 2.25

WINDOW 2/9: Training on rounds 1-26


...

Window 8 Results:
  MAE: 0.826
  Log Likelihood: -43.57 (vs Naive: -46.35)
  Improvement: 2.78

WINDOW 9/9: Training on rounds 1-33


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (3 chains in 3 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]


Output()

Sampling 3 chains for 2_000 tune and 2_000 draw iterations (6_000 + 6_000 draws total) took 238 seconds.
There were 708 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Window 9 Results:
  MAE: 0.804
  Log Likelihood: -66.13 (vs Naive: -71.17)
  Improvement: 5.04

CROSS-VALIDATION COMPLETE


In [7]:
# Summarize cross-validation results
results_df = pd.DataFrame(results)

print("\n=== CROSS-VALIDATION SUMMARY ===\n")
print(results_df.to_string(index=False))

print(f"\n{'='*60}")
print("AVERAGE PERFORMANCE ACROSS ALL WINDOWS")
print(f"{'='*60}")
print(f"Mean MAE: {results_df['mae'].mean():.3f} ± {results_df['mae'].std():.3f}")
print(f"Mean Log Likelihood: {results_df['log_likelihood'].mean():.2f} ± {results_df['log_likelihood'].std():.2f}")
print(f"Mean LL Improvement over Naive: {results_df['ll_improvement'].mean():.2f} ± {results_df['ll_improvement'].std():.2f}")
print(f"Mean beta_xG: {results_df['beta_xG'].mean():.3f} ± {results_df['beta_xG'].std():.3f}")

print(f"\n{'='*60}")
print("PERFORMANCE STABILITY")
print(f"{'='*60}")
print(f"MAE Range: {results_df['mae'].min():.3f} to {results_df['mae'].max():.3f}")
print(f"LL Improvement Range: {results_df['ll_improvement'].min():.2f} to {results_df['ll_improvement'].max():.2f}")
print(f"All windows beat naive baseline: {(results_df['ll_improvement'] > 0).all()}")


=== CROSS-VALIDATION SUMMARY ===

 window train_rounds test_rounds  n_train  n_test      mae  log_likelihood    ll_naive  ll_improvement            beta_xG
      1         1-25       26-26      562      32 0.896210      -50.045064  -46.795610       -3.249455 0.8650217598855807
      2         1-26       27-27      578      80 0.658699     -101.039836 -109.964349        8.924513 0.8396239054658629
      3         1-27       28-28      618      40 0.731261      -53.626309  -66.613612       12.987303 0.8341806054267192
      4         1-28       29-29      638      44 1.059551      -64.342863  -74.219989        9.877126 0.8419215369803251
      5         1-29       30-30      660      36 0.630198      -46.831527  -57.006277       10.174749 0.8380414819807616
      6         1-30       31-31      678      44 0.648525      -55.748766  -62.075977        6.327211 0.8291160317201468
      7         1-31       32-32      700      40 0.727783      -50.019390  -56.653447        6.634057 0.836635